In [ ]:
# http://ecomunsing.com/build-your-own-blockchain

In [1]:
import hashlib
import json
import sys

In [2]:
# Encrypt information:

def hashMe(msg=""):
    # Helper function to wrap hashing alg.
    
    if type(msg) != str:
        msg = json.dumps(msg,sort_keys=True)
        # If we don't sort keys, we can't guarantee repetability.
        
    if sys.version_info.major == 2:
        return unicode(hashlib.sha256(msg).hexdigest(), 'utf-8')
    else:
        return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()
        

In [3]:
import random
random.seed(0)

In [4]:
# Create transactions :

def makeTransaction(maxValue=3):
    # Create valid transactions in range(1, maxValue)
    
    # Randomly choose -1 or 1:
    sign = int(random.getrandbits(1))*2 - 1
    
    amount = random.randint(1,maxValue)
    alicePays = sign * amount
    bobPays = -1* alicePays
    
    return{u'Alice' : alicePays,
          u'Bob' : bobPays}
    
    # By construction, this will always return transactions that
    # respect the conservation of tokens.
    
    # Note : We have not checked for overdraft.
    

In [76]:
# Create a large set of transactions to hold in blocks :

txnBuffer = [makeTransaction() for i in range(30)]

In [77]:
# In Bitcoin, the validation function checks that :

# 1.The input values are valid unspent transaction outputs (UTXOs)
# 2.The output of the transactions are no greater than the input
# 3.The keys used for the signatures are valid

# In Etherium, the validation function checks that the smart
# contracts were faithfully executed and respect gas limit

In [78]:
# The rules we will implement :

#    1. The sum of the deposits and withdrawls must be 0
#       (tolkens are neither created nor destroyed)
#    2. A user;s account must have sufficient funds.

# If either conditions are violated, reject the transaction.

In [79]:
def updateState(txn, state):
    # Inputs :
    # dictionaries holding account names and :
    # txn : numeric values for transfer amount
    # state : account balance
    
    # Returns :
    # Updated state
    
    # Note : This does not validate the transaction, just update
    # state.
    
    state = state.copy()
    
    for key in txn:
        if key in state.keys():
            state[key] += txn[key]
        else:
            state[key] = txn[key]
    
    return state

In [80]:
# Validate transaction :

def isValidTxn(txn,state):
    # Assume that the transaction is a dictionary keyed by account names

    # Check that the sum of the deposits and withdrawals is 0
    if sum(txn.values()) != 0:
        return False
    
    # Check that the transaction does not cause an overdraft
    for key in txn.keys():
        if key in state.keys(): 
            acctBalance = state[key]
        else:
            acctBalance = 0
        if (acctBalance + txn[key]) < 0:
            return False
    
    return True

In [81]:
# Example transactions, some fraudulent :

state = {u'Alice':5,u'Bob':5}

print(isValidTxn({u'Alice': -3, u'Bob': 3},state))  # Basic transaction- this works great!
print(isValidTxn({u'Alice': -4, u'Bob': 3},state))  # But we can't create or destroy tokens!
print(isValidTxn({u'Alice': -6, u'Bob': 6},state))  # We also can't overdraft our account.
print(isValidTxn({u'Alice': -4, u'Bob': 2,'Lisa':2},state)) # Creating new users is valid
print(isValidTxn({u'Alice': -4, u'Bob': 3,'Lisa':2},state)) # But the same rules still apply!

True
False
False
True
False


In [82]:
# Genesis Block :
state = {u'Alice':50, u'Bob':50}

genesisBlockTxns = [state]
genesisBlockContents = {u'blockNumber' : 0,
                        u'parentHash': None,
                        u'txnCount' : 1,
                        u'txns' : genesisBlockTxns}

genesisHash = hashMe(genesisBlockContents)

#print(genesisHash)

genesisBlock = { u'hash' : genesisHash,
                 u'contents' : genesisBlockContents}

genesisBlockStr = json.dumps(genesisBlock, sort_keys=True, indent=2)

print(genesisBlockStr)


{
  "contents": {
    "blockNumber": 0,
    "parentHash": null,
    "txnCount": 1,
    "txns": [
      {
        "Alice": 50,
        "Bob": 50
      }
    ]
  },
  "hash": "7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507"
}


In [83]:
chain = [genesisBlock]
print(chain)

[{'hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507', 'contents': {'blockNumber': 0, 'parentHash': None, 'txnCount': 1, 'txns': [{'Alice': 50, 'Bob': 50}]}}]


In [84]:
# For each block we want to collect a set of transactions, 
# create a header, hash it and add it to the chain.

In [85]:
def makeBlock(txns, chain):
    parentBlock = chain[-1]
    parentHash = parentBlock[u'hash']
    blockNumber = parentBlock[u'contents'][u'blockNumber'] + 1
    txnCount = len(txns)
    
    blockContents = {u'blockNumber' : blockNumber,
                    u'parentHash' : parentHash,
                    u'txnCount' : len(txns),
                    'txns' : txns}
    
    blockHash = hashMe(blockContents)
    block = {u'hash' : blockHash,
             u'contents' : blockContents}
    
    return block

In [86]:
# Use it to process the transaction buffer from earlier :

blockSizeLimit = 5 # Transactions per block, chosen by the miner.
                   # Can vary beween blocks.
    
while len(txnBuffer) > 0:
    bufferStartSize = len(txnBuffer)
    
    # Get a set of valid transactions :
    txnList = []
    while (len(txnBuffer) > 0) & (len(txnList) < blockSizeLimit):
        newTxn = txnBuffer.pop()
        validTxn = isValidTxn(newTxn, state) # Return false if invalid
        
        if validTxn:
            txnList.append(newTxn)
            state = updateState(newTxn, state)
        else:
            print("ignored transaction")
            sys.stdout.flush()
            continue
            
    # Make a block
    myBlock = makeBlock(txnList, chain)
    chain.append(myBlock)

In [87]:
print(chain)

[{'hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507', 'contents': {'blockNumber': 0, 'parentHash': None, 'txnCount': 1, 'txns': [{'Alice': 50, 'Bob': 50}]}}, {'hash': 'b42642d5eb64a407d990685ec113713ee4d64daf8b7296f852b99f3a26e1432d', 'contents': {'blockNumber': 1, 'parentHash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507', 'txnCount': 5, 'txns': [{'Alice': 1, 'Bob': -1}, {'Alice': -2, 'Bob': 2}, {'Alice': 3, 'Bob': -3}, {'Alice': -1, 'Bob': 1}, {'Alice': -3, 'Bob': 3}]}}, {'hash': 'fbf4ef63adeb70bc52da269def7d00aa78532b1619454bc47aa91e2be812ac4e', 'contents': {'blockNumber': 2, 'parentHash': 'b42642d5eb64a407d990685ec113713ee4d64daf8b7296f852b99f3a26e1432d', 'txnCount': 5, 'txns': [{'Alice': 2, 'Bob': -2}, {'Alice': 2, 'Bob': -2}, {'Alice': 3, 'Bob': -3}, {'Alice': 3, 'Bob': -3}, {'Alice': 2, 'Bob': -2}]}}, {'hash': '357974bff0a1ad1b8119b6f8a54a72b291a87466ffefa03dd53b28824d9e178f', 'contents': {'blockNumber': 3, 'parentHash': 'fbf4ef63ade

In [88]:
chain[0]

{'hash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
 'contents': {'blockNumber': 0,
  'parentHash': None,
  'txnCount': 1,
  'txns': [{'Alice': 50, 'Bob': 50}]}}

In [89]:
chain[1]

{'hash': 'b42642d5eb64a407d990685ec113713ee4d64daf8b7296f852b99f3a26e1432d',
 'contents': {'blockNumber': 1,
  'parentHash': '7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507',
  'txnCount': 5,
  'txns': [{'Alice': 1, 'Bob': -1},
   {'Alice': -2, 'Bob': 2},
   {'Alice': 3, 'Bob': -3},
   {'Alice': -1, 'Bob': 1},
   {'Alice': -3, 'Bob': 3}]}}

In [90]:
state

{'Alice': 43, 'Bob': 57}

In [372]:

# Let's define functinos to check if the new blocks are valid
# and if the whole chain is valid

# On a blockchain network, this becomes important in two ways :
#   * When we initially set up our node, we will download the
#     full blockchain history. After that we would need to run
#     through the blockchain to compute the state of the system.
#     To protect against inserting invalid transactions in the 
#     initial chain, we need to check the validity of the entire
#     chain in the initial download.

#   * Once out node is synced with the network (has and up-to-date
#     copy of the blockchain and representation of system state)
#     it will need to check the validity of new blocks that are
#      broadcast to the network.

In [285]:

# We will need three functions to facilitate this :

# checkBlockHash : A simple helper function that makes sure that
#                  the block contents match the hash.

# checkBlockValidity : Check the validity of the block, given its 
#                      parent and the current system state. We
#                      will need to return the updated state if
#                      the block is valid, and raise an error 
#                      otherwise.

# checkChain : Check the validity of the entire chain, and compute
#              the system state beginning at the genesis block.
#              This will return the system state if the chain is
#              valid, and raise an error otherwise.

In [93]:
def checkBlockHash(block):
    expectedHash = hashMe(block['contents'])
    
    if block['hash']!= expectedHash:
        raise Exception('Hash does not match contents of block %s'%block['contents']['blockNumber'])
               
    return

In [67]:
def checkBlockValidity(block, parent, state):
    # We want to check :
    #    * Each of the transactions are valid updates to the
    #      system state.    
    #    * Block hash is valid for block contents.
    #    * Block number increments the parent block number by 1.
    #    * Accurately reference the parent block hash.
    
    parentNumber = parent['contents']['blockNumber']
    parentHash   = parent['hash']
    blockNumber  = block['contents']['blockNumber']

    # Check transaction validity :
    for txn in block['contents']['txns']:
        if isValidTxn(txn,state):
            state = updateState(txn,state)
        else:
            raise Exception('Invaid transaction in block %s: %s' %(blockNumber, txn))
            
    # Check block hash:
    checkBlockHash(block)
    
    # Check block number :
    if blockNumber !=(parentNumber+1):
        raise Exception('Hash does not match contents of block %s' %blockNumber)
        
    # Check parent hash storage in block:
    if block['contents']['parentHash'] != parentHash:
        raise Exception('Parent hash not accurate at block %s'%blockNumber)
    
    return state

In [95]:
def checkChain(chain):
    # Work through the chain from genesis block
    # Check :
    #    * All transactions are internally valid.
    #    * Blocks are linked by hashes.
    #    * Transaction does not cause overdraft
    
    # This returns the state as a dictionary of accounts and
    # balances or returns False.
    
    
    # Make sure that the chain is a list of dicts:
    if type(chain)==str:
        try:
            chain = json.loads(chain)
            assert( type(chain)==list)
        except :
            return False
    elif type(chain) != list:
        return False
    
    state = {}

    # Special treatment for genesis block :
    for txn in chain[0]['contents']['txns']:
        state = updateState(txn,state)
    checkBlockHash(chain[0])
    parent = chain[0]
    
    # Subsequent blocks :
    for block in chain[1:]:
        state = checkBlockValidity(block, parent,state)
        parent = block
        
    return state

In [96]:
checkChain(chain)

{'Alice': 43, 'Bob': 57}

In [98]:
chainAsText = json.dumps(chain,sort_keys=True, indent=2)
print(chainAsText)

[
  {
    "contents": {
      "blockNumber": 0,
      "parentHash": null,
      "txnCount": 1,
      "txns": [
        {
          "Alice": 50,
          "Bob": 50
        }
      ]
    },
    "hash": "7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507"
  },
  {
    "contents": {
      "blockNumber": 1,
      "parentHash": "7c88a4312054f89a2b73b04989cd9b9e1ae437e1048f89fbb4e18a08479de507",
      "txnCount": 5,
      "txns": [
        {
          "Alice": 1,
          "Bob": -1
        },
        {
          "Alice": -2,
          "Bob": 2
        },
        {
          "Alice": 3,
          "Bob": -3
        },
        {
          "Alice": -1,
          "Bob": 1
        },
        {
          "Alice": -3,
          "Bob": 3
        }
      ]
    },
    "hash": "b42642d5eb64a407d990685ec113713ee4d64daf8b7296f852b99f3a26e1432d"
  },
  {
    "contents": {
      "blockNumber": 2,
      "parentHash": "b42642d5eb64a407d990685ec113713ee4d64daf8b7296f852b99f3a26e1432d",
      "tx

In [99]:
# Node A:
import copy
nodeBchain = copy.copy(chain)
nodeBtxns  =[makeTransaction() for i in range(5)]
newBlock   = makeBlock(nodeBtxns, nodeBchain)

In [100]:
# Now assume that the new block is transmitted into our node
# and we need to check it and update our state.

In [101]:
print("Blockchain on Node A is currently %s block long"%len(chain))

try:
    print("New Block Recieved; Checking validity . . .")
    state = checkBlockValidity(newBlock,chain[-1],state)
    
    # Update the state (will throw error if block is invalid) :
    chain.append(newBlock)

except:
    print("Invalid block.")

print("Blockchain on node A is now %s blocks long"%len(chain))

Blockchain on Node A is currently 7 block long
New Block Recieved; Checking validity . . .
Blockchain on node A is now 8 blocks long
